In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
import pandas as pd
import re
import spacy

In [3]:
import zipfile
import pandas as pd
import numpy as np
import re
from pathlib import Path
path = './'
zip_path = "data/Datos.zip"
with zipfile.ZipFile(zip_path) as z:
    # ajusta nombres si difieren
    movies_df = pd.read_csv(z.open('rotten_tomatoes_movies.csv'), low_memory=False)
    reviews_df = pd.read_csv(z.open('rotten_tomatoes_critic_reviews.csv'), low_memory=False)

movies_df.shape, reviews_df.shape


((17712, 22), (1130017, 8))

In [4]:
movies_df

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,...,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,...,20th Century Fox,Rotten,49.0,149.0,Spilled,53.0,254421.0,43,73,76
1,m/0878835,Please Give,Kate (Catherine Keener) and her husband Alex (...,Nicole Holofcener's newest might seem slight i...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",2010-04-30,...,Sony Pictures Classics,Certified-Fresh,87.0,142.0,Upright,64.0,11574.0,44,123,19
2,m/10,10,"A successful, middle-aged Hollywood songwriter...",Blake Edwards' bawdy comedy may not score a pe...,R,"Comedy, Romance",Blake Edwards,Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ...",1979-10-05,...,Waner Bros.,Fresh,67.0,24.0,Spilled,53.0,14684.0,2,16,8
3,m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),Following the closing arguments in a murder tr...,Sidney Lumet's feature debut is a superbly wri...,NR,"Classics, Drama",Sidney Lumet,Reginald Rose,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",1957-04-13,...,Criterion Collection,Certified-Fresh,100.0,54.0,Upright,97.0,105386.0,6,54,0
4,m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea","In 1866, Professor Pierre M. Aronnax (Paul Luk...","One of Disney's finest live-action adventures,...",G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,Earl Felton,"James Mason, Kirk Douglas, Paul Lukas, Peter L...",1954-01-01,...,Disney,Fresh,89.0,27.0,Upright,74.0,68918.0,5,24,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17707,m/zoot_suit,Zoot Suit,Mexican-American gangster Henry Reyna (Daniel ...,NaN,R,"Drama, Musical & Performing Arts",Luis Valdez,Luis Valdez,"Daniel Valdez, Edward James Olmos, Charles Aid...",1981-10-02,...,MCA Universal Home Video,Rotten,56.0,9.0,Upright,74.0,1195.0,2,5,4
17708,m/zootopia,Zootopia,From the largest elephant to the smallest shre...,The brilliantly well-rounded Zootopia offers a...,PG,"Action & Adventure, Animation, Comedy","Byron Howard, Rich Moore, Jared Bush","Jared Bush, Phil Johnston","J.K. Simmons, Kristen Bell, Octavia Spencer, A...",2016-03-04,...,Walt Disney Animation Studios,Certified-Fresh,98.0,291.0,Upright,92.0,101511.0,50,285,7
17709,m/zorba_the_greek,Zorba the Greek,Traveling to inspect an abandoned mine his fat...,NaN,NR,"Action & Adventure, Art House & International,...",NaN,NaN,"Anthony Quinn, Alan Bates, Irene Papas, Lila K...",1964-12-17,...,Fox,Fresh,80.0,10.0,Upright,86.0,7146.0,0,8,2
17710,m/zulu,Zulu,"In 1879, the Zulu nation hands colonial Britis...",Zulu patiently establishes a cast of colorful ...,PG,"Classics, Drama","Cy Endfield, Cyril Endfield","Cy Endfield, John Prebble","Stanley Baker, Jack Hawkins, Ulla Jacobsson, J...",1964-06-17,...,Paramount Pictures,Fresh,96.0,23.0,Upright,91.0,30193.0,6,22,1


In [5]:
reviews_df

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...
1,m/0814255,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,m/0814255,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...
4,m/0814255,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...
...,...,...,...,...,...,...,...,...
1130012,m/zulu_dawn,Chuck O'Leary,False,Fantastica Daily,Rotten,2/5,2005-11-02,NaN
1130013,m/zulu_dawn,Ken Hanke,False,"Mountain Xpress (Asheville, NC)",Fresh,3.5/5,2007-03-07,"Seen today, it's not only a startling indictme..."
1130014,m/zulu_dawn,Dennis Schwartz,False,Dennis Schwartz Movie Reviews,Fresh,B+,2010-09-16,A rousing visual spectacle that's a prequel of...
1130015,m/zulu_dawn,Christopher Lloyd,False,Sarasota Herald-Tribune,Rotten,3.5/5,2011-02-28,"A simple two-act story: Prelude to war, and th..."


In [6]:
# 1. Agrupar los comentarios de los críticos
# Usamos 'rotten_tomatoes_link' como ID de la película
reviews_aggregated = reviews_df.groupby('rotten_tomatoes_link')['review_content'].apply(lambda x: ' '.join(x.astype(str))).reset_index()
reviews_aggregated.rename(columns={'review_content': 'aggregated_reviews'}, inplace=True)

print("Shape del DataFrame de revisiones agregadas:", reviews_aggregated.shape)
print("Ejemplo de revisiones agregadas:")
print(reviews_aggregated.head())

# 2. Unir el DataFrame de películas con las revisiones agregadas
# Realizamos un LEFT JOIN para mantener todas las películas
corpus_df = pd.merge(movies_df, reviews_aggregated, on='rotten_tomatoes_link', how='left')

# 3. Crear el campo de corpus combinado

corpus_df['description'] = corpus_df['movie_info'].fillna('')
corpus_df['aggregated_reviews'] = corpus_df['aggregated_reviews'].fillna('')

# Corpus final para el SRI: Descripción + Comentarios del Crítico
corpus_df['corpus'] = corpus_df['description'] + ' ' + corpus_df['aggregated_reviews']
corpus_df['doc_id'] = corpus_df.index # ID único para el ranking

print("\nShape del Corpus Combinado (Películas + Reviews):", corpus_df.shape)
print("\nColumnas del Corpus Final (incluyendo el texto combinado):")
print(corpus_df[['doc_id', 'movie_title', 'description', 'aggregated_reviews', 'corpus']].head(3).to_markdown(index=False))

Shape del DataFrame de revisiones agregadas: (17712, 2)
Ejemplo de revisiones agregadas:
              rotten_tomatoes_link  \
0                     m/+_one_2019   
1                             m/+h   
2                          m/-_man   
3  m/-cule_valley_of_the_lost_ants   
4                        m/0814255   

                                  aggregated_reviews  
0  More than gets by on the strength of its funda...  
1  Ultimately Plush exceeds its very limited expe...  
2  nan Too many pieces of the puzzle are left out...  
3  A dialogue-free bug saga carried along by bril...  
4  A fantasy adventure that fuses Greek mythology...  

Shape del Corpus Combinado (Películas + Reviews): (17712, 26)

Columnas del Corpus Final (incluyendo el texto combinado):
|   doc_id | movie_title                                        | description                                                                                                                                                        

In [11]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
import pandas as pd
import re
import spacy

# 1. Cargar recursos de NLTK (si no se han cargado antes)
# nltk.download('punkt')
# nltk.download('stopwords')

# 2. Inicializar recursos
# Para Stemming: SnowballStemmer es ideal para español
stemmer = SnowballStemmer('spanish')
stop_words = set(stopwords.words('spanish'))
nlp = spacy.load("es_core_news_sm")
print("Modelo SpaCy cargado correctamente. ¡Listos para lematizar!")

OSError: [E050] Can't find model 'es_core_news_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [40]:
def full_preprocess(text, method='lemma'):
    if not isinstance(text, str):
        return []

    # 1. Tokenización y Normalización (a minúsculas)
    tokens = word_tokenize(text.lower(), language='spanish')

    # 2. Filtrado y Limpieza
    # Eliminar palabras que no son alfabéticas (puntuación, números) y stop words
    filtered_tokens = [token for token in tokens if token.isalpha() and token not in stop_words]

    # 3. Aplicación de Lematización o Stemming
    if method == 'lemma':
        # Lematización con SpaCy (manteniendo el significado léxico)
        # Se requiere un procesamiento adicional para pasar los tokens a SpaCy
        doc = spacy.tokens.Doc(nlp.vocab, words=filtered_tokens)
        doc = nlp.make_doc(doc.text)
        processed_tokens = [token.lemma_ for token in doc]

    elif method == 'stem':
        # Stemming con SnowballStemmer (raíz morfológica)
        processed_tokens = [stemmer.stem(token) for token in filtered_tokens]

    else:
        # Si el método no es reconocido, retorna solo los tokens filtrados
        processed_tokens = filtered_tokens

    return processed_tokens

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Inicializar y ajustar el vectorizador TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(corpus_df['processed_text'])

def get_cosine_scores(query_text, matrix, vectorizer, df):
    """Calcula la similitud coseno entre la consulta y todos los documentos."""
    # Usar la función de preprocesamiento de lematización en la consulta
    processed_query = ' '.join(full_preprocess(query_text, method='lemma'))

    if not processed_query:
        return pd.DataFrame({'doc_id': df['doc_id'], 'title': df['movie_title'], 'relevance_score': 0.0})

    query_vector = vectorizer.transform([processed_query])
    cosine_scores = cosine_similarity(query_vector, matrix).flatten()

    results = pd.DataFrame({
        'doc_id': df['doc_id'],
        'title': df['movie_title'],
        'relevance_score': cosine_scores
    }).sort_values(by='relevance_score', ascending=False).reset_index(drop=True)

    return results

In [ ]:
def jaccard_similarity(set1, set2):
    """Calcula la similitud Jaccard."""
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

def get_jaccard_scores(query_text, df):
    """Calcula la similitud Jaccard entre la consulta y cada documento."""
    query_tokens = set(full_preprocess(query_text, method='lemma'))

    if not query_tokens:
        return pd.DataFrame({'doc_id': df['doc_id'], 'title': df['movie_title'], 'relevance_score': 0.0})

    jaccard_scores = []
    for tokens in df['processed_tokens']:
        doc_tokens = set(tokens)
        score = jaccard_similarity(query_tokens, doc_tokens)
        jaccard_scores.append(score)

    results = pd.DataFrame({
        'doc_id': df['doc_id'],
        'title': df['movie_title'],
        'relevance_score': jaccard_scores
    }).sort_values(by='relevance_score', ascending=False).reset_index(drop=True)

    return results

In [ ]:
consulta_1 = "Películas sobre viajes espaciales."

print(f"--- 🚀 Resultados para: '{consulta_1}' (Top 10) ---")

# 1. TF-IDF / Coseno
result_tfidf = get_cosine_scores(consulta_1, tfidf_matrix, vectorizer, corpus_df)
print("\n[TF-IDF / Coseno]")
print(result_tfidf[['title', 'relevance_score']].head(10))

# 2. Jaccard
result_jacard = get_jaccard_scores(consulta_1, corpus_df)
print("\n[Jaccard]")
print(result_jacard[['title', 'relevance_score']].head(10))

# 3. BM25
result_bm25 = get_bm25_scores(consulta_1, corpus_df)
print("\n[BM25]")
print(result_bm25[['title', 'relevance_score']].head(10))

In [ ]:
# DEFINICIÓN MANUAL DEL GROUND TRUTH (Ejemplo)
# Asume que estos IDs (índices) son relevantes para "Viajes Espaciales"
ids_relevantes_ejemplo = [42, 105, 501, 890, 1500, 2100, 3050, 4100, 5200, 6000]
k = 10
total_relevantes = len(ids_relevantes_ejemplo)


def calcular_precision_recall(resultados_sistema: pd.DataFrame, ids_relevantes: List[int], k: int = 10) -> Dict[
    str, float]:
    """Calcula Precision@K y Recall@K."""

    top_k_results = resultados_sistema.head(k)['doc_id'].tolist()
    set_ids_relevantes = set(ids_relevantes)

    relevantes_recuperados = sum(1 for doc_id in top_k_results if doc_id in set_ids_relevantes)

    precision_at_k = relevantes_recuperados / k if k > 0 else 0
    total_relevantes = len(ids_relevantes)
    recall_at_k = relevantes_recuperados / total_relevantes if total_relevantes > 0 else 0

    return {
        'precision@k': precision_at_k,
        'recall@k': recall_at_k
    }


print(f"\n## Evaluación de Modelos para '{consulta_1}' (K={k}) ##")
print(f"Total de documentos relevantes simulados: {total_relevantes}\n")

metricas = calcular_precision_recall(result_jacard, ids_relevantes_ejemplo, k=k)
print(f"**Jaccard** -> Precisión@{k}: {metricas['precision@k']:.4f}, Recall@{k}: {metricas['recall@k']:.4f}")
metricas = calcular_precision_recall(result_tfidf, ids_relevantes_ejemplo, k=k)
print(f"**TF-IDF/Coseno** -> Precisión@{k}: {metricas['precision@k']:.4f}, Recall@{k}: {metricas['recall@k']:.4f}")
metricas = calcular_precision_recall(result_bm25, ids_relevantes_ejemplo, k=k)
print(f"**BM25** -> Precisión@{k}: {metricas['precision@k']:.4f}, Recall@{k}: {metricas['recall@k']:.4f}")